In [ ]:
train_dir = '//input/brain-tumor-mri-dataset/Training/'
test_dir = '/input/brain-tumor-mri-dataset/Testing/'

In [ ]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data = keras.utils.image_dataset_from_directory(
    train_dir,
    validation_split = None,
    image_size = (256,256),
    batch_size = 32,
    labels = 'inferred',
    label_mode = 'categorical'
)

In [ ]:
len(train_data)

In [ ]:
test_data,val_data = keras.utils.image_dataset_from_directory(
    test_dir,
    
    validation_split = 0.5,
    subset = 'both',
    image_size = (256,256),
    seed = 1223,
    batch_size = 32,
    labels = 'inferred',
    label_mode = 'categorical'
)

In [ ]:
len(train_data), len(test_data), len(val_data)

In [ ]:
classes = train_data.class_names

In [ ]:
plt.figure(figsize = (10,10))
for img , label in train_data.take(1):
    for i in range(0,9):
        plt.subplot(3,3,i+1)
        
        plt.imshow(img[i].numpy().astype('uint'))
        plt.title(classes[np.argmax(label[i])])
        plt.axis('off')
        
        

In [ ]:
resize_and_rescale = keras.Sequential([
    keras.layers.Resizing(256,256),
    keras.layers.Rescaling(1./255)
])

In [ ]:

data_aug = keras.Sequential([
    keras.layers.RandomRotation(0.2),
    keras.layers.RandomFlip(),
    keras.layers.RandomTranslation(height_factor = 0.1, width_factor = 0.2),
    keras.layers.RandomContrast(0.1)
])

In [ ]:
from tensorflow.keras import backend as K

K.clear_session()

In [ ]:
from keras.applications import Xception
base_model = Xception(include_top = False, weights = 'imagenet', pooling = 'max')

In [ ]:
img_shape = (256,256,3)

In [ ]:
model = keras.Sequential([
    keras.layers.Input(shape = img_shape),
    resize_and_rescale, 
    base_model, 
    keras.layers.Flatten(),
    keras.layers.Dropout(rate= 0.3),
    keras.layers.Dense(128, activation= 'relu'),
    keras.layers.Dropout(rate= 0.25),
    keras.layers.Dense(64, activation = 'relu'),
    keras.layers.Dropout(rate = 0.15),
    keras.layers.Dense(4, activation= 'softmax')
])


In [ ]:
model.summary()  

In [ ]:
model.compile(optimizer = 'Adam', loss = 'categorical_crossentropy', metrics = ['accuracy','precision', 'recall'])

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(
    monitor = 'val_accuracy',
    patience = 2,
    restore_best_weights = True
)

In [ ]:
hist = model.fit(train_data,
          validation_data = val_data , 
          epochs = 10,
          batch_size = 32,
          callbacks = [early_stop])

In [ ]:
model.evaluate(test_data, verbose = 1)

In [ ]:
history = hist

In [ ]:
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

# Plot training & validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
model.save('mrimodel.keras')

In [ ]:
load_model = keras.models.load_model('mrimodel.keras')

In [ ]:
load_model.summary()

In [ ]:
load_model.evaluate(test_data, verbose = 1)